In [24]:
# first convert ods to csv

import pandas as pd

read in inflections table

In [25]:
inflection_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection-generator/declensions & conjugations.xlsx', sheet_name="declensions", dtype=str)

inflection_df = inflection_df.shift(periods=2)

inflection_df.columns = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "AC", "AD", "AE", "AF", "AG", "AH", "AI", "AJ", "AK", "AL", "AM", "AN", "AO", "AP", "AQ", "AR", "AS", "AT", "AU", "AV", "AW", "AX", "AY", "AZ", "BA", "BB", "BC", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BK", "BL", "BM", "BN", "BO", "BP", "BQ", "BR", "BS", "BT", "BU", "BV", "BW", "BX", "BY", "BZ", "CA", "CB", "CC", "CD", "CE", "CF", "CG", "CH", "CI", "CJ", "CK", "CL", "CM", "CN", "CO", "CP", "CQ", "CR", "CS", "CT", "CU", "CV", "CW", "CX", "CY", "CZ", "DA", "DB", "DC", "DD", "DE", "DF", "DG", "DH", "DI", "DJ", "DK"]
inflection_df.fillna("", inplace=True)

read index

In [26]:
index_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection-generator/declensions & conjugations.xlsx', sheet_name="index", dtype=str)
index_df.fillna("", inplace=True)
# index_df["inflection name"] = index_df["inflection name"].str.replace(" ", "_") #only needed if using as variable
index_df_length = len(index_df)

generate df's of each inflection pattern

In [27]:
import re

for row in range(index_df_length):
	inflection_name = index_df.iloc[row,0]
	cell_range = index_df.iloc[row,1]
	like = index_df.iloc[row,2]
	irreg = index_df.iloc[row,3]
	
	col_range_1 = re.sub("(.+?)\d*\:.+", "\\1", cell_range)
	col_range_2 = re.sub(".+\:(.[A-Z]*)\d*", "\\1", cell_range)
	row_range_1 = int(re.sub(".+?(\d{1,3}):.+", "\\1", cell_range))
	row_range_2 = int(re.sub(".+:.+?(\d{1,3})", "\\1", cell_range))

	# print (f"{inflection_name} || {cell_range} || {col_range_1}:{col_range_2} || {row_range_1}:{row_range_2}")

	inflection_df_filtered = inflection_df.loc[row_range_1:row_range_2, col_range_1:col_range_2]
	inflection_df_filtered.Name =  f"{inflection_name}" #help! change the name to "inflection name" 

	inflection_df_filtered.to_csv(f"output/patterns/{inflection_name}.csv", sep="\t", index=False, header=False)

generate csv for each headword

In [28]:
dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd.csv", sep="\t", dtype=str)
dpd_df.fillna("", inplace=True)
dpd_df_length = len(dpd_df)

all_inflections = ""

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	headword_clean = re.sub(" \d*$", "", headword)
	stem = dpd_df.loc[row, "Stem"]
	if stem == "*":
		stem = ""
	pattern = dpd_df.loc[row, "Pattern"]
	pos = dpd_df.loc[row, "POS"]
	metadata = dpd_df.loc[row, "Metadata"]
	meaning = dpd_df.loc[row, "Meaning IN CONTEXT"]

	if row % 2000 == 0:
		print(f"{row} {headword}")

	if pos != "prefix" and pos != "cs" and pos != "suffix" and pos != "ve" and pos != "idiom" and meaning != "" and metadata != "yes":
		# print(f"{headword=} {headword_clean=} {stem=} {pattern=} {pos=} {metadata=} {meaning=}")

		if stem == "-":
			all_inflections += headword_clean + " "

		elif stem == "!":
			all_inflections += headword_clean + " "

		else:
			try:
				df = pd.read_csv(f"output/patterns/{pattern}.csv", sep="\t", header=None)
				df.fillna("", inplace=True)
				df_rows = df.shape[0]
				df_columns = df.shape[1]

				for rows in range(1, df_rows):
					for columns in range(1, df_columns, 2):
						line = df.iloc[rows, columns]
						if line == "sg" or line == "pl":
							pass

						else:
							line = re.sub(r"(.+)", f"{stem}\\1", line)
							search_string = re.compile("\n", re.M)
							replace_string = " "
							matches = re.sub(search_string, replace_string, line)

							all_inflections += matches + " "
						
			except:
				with open("output/errors.txt", "a") as error_log:
					error_log.write(f"error on: {headword}\n")

0 ā
1000 accimant 2
2000 ajapāla
3000 āṇa
4000 anucara 2
5000 aparihāna
6000 āsaṅkitaparisaṅkita
7000 avabodha 1
8000 bālavasanta
9000 bindava
10000 chanda 2
11000 dāyajja 1
12000 dubbalaṭṭhāna
13000 evarūpopi
14000 gumba 2
15000 iṭṭhikā
16000 kāḷī
17000 kathaṃdassī
18000 kiṃsamācāra
19000 līna 1
20000 mandavāta
21000 mūsikā
22000 nevasaññānāsaññāyatanasamāpatti
23000 nirantara
24000 otāraṇa
25000 pajjhāyanta
26000 papañcayanta
27000 parisussi
28000 paṭiladdhapaccaya
29000 petaloka
30000 pūrenta
31000 sabbayogavisaṃyutta
32000 sāmañña 6
33000 saṃsīdana
34000 saññāpaññatti
35000 sāyanīya 1
36000 sudinna
37000 tañhi
38000 tiṇadosa
39000 ukkoṭitita
40000 uppajjanaudaka
41000 vante
42000 vigarahanta
43000 vippakiṇṇa 2
44000 vuṭṭhāpeti


In [29]:
all_inflections_list = all_inflections.split()
all_inflections_list = list(dict.fromkeys(all_inflections_list))
all_inflections_df = pd.DataFrame(all_inflections_list)
all_inflections_df.to_csv("output/all_inflections_df.csv", index=False, header=False)